In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd ='C:/Program Files/Tesseract-OCR/tesseract.exe'
from googletrans import Translator
from langdetect import detect
import re

In [6]:
def remove_special_characters(text):
    # Define the pattern for special characters
    pattern = r'[^a-zA-Z0-9\s]'
    
    # Remove special characters using regex
    text = re.sub(pattern, '', text)
    
    return text

In [7]:
def get_first_word(text):
    # Split the text into words
    words = text.split()
    
    # Retrieve the first word
    if len(words) > 0:
        first_word = words[0]
        return first_word
    
    # Return None if the text is empty or contains no words
    return None

In [8]:
input_string= 'D:/Thesis-20230616T142242Z-001/Thesis/Pic/8.jpg'
dataset_csv = 'D:/Thesis-20230616T142242Z-001/Thesis/medicine..csv'
dataset_image = 'D:/Thesis-20230616T142242Z-001/Thesis/image_dataset'

In [9]:
image = Image.open(input_string)

In [10]:
textOfImage = pytesseract.image_to_string(image, lang='eng') # for all latin languages
# textOfImage = pytesseract.image_to_string(image, lang='chi_sim') #for non-latin chinease languages 
clean_text = remove_special_characters(textOfImage)
print(get_first_word(clean_text))
text_after_word= get_first_word(clean_text)
# english_string = convert_to_english(textOfImage)
print(text_after_word)

Seclo
Seclo


In [88]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import pyttsx3

# Read the dataset
dataset_path = dataset_csv
data = pd.read_csv(dataset_path)

# Drop rows with missing values
data = data.dropna()

# Separate X and y
X = data['brand name']
y = data[['type', 'slug', 'dosage form', 'generic','manufacturer','package container']]

# Initialize label encoders and encode y
label_encoders = {}
y_encoded = {}

for col in y.columns:
    label_encoders[col] = LabelEncoder()
    y_encoded[col] = label_encoders[col].fit_transform(y[col])

# Convert y_encoded to a DataFrame and reset indices to align X and y
y_encoded = pd.DataFrame(y_encoded)
X = X.reset_index(drop=True)
y_encoded = y_encoded.reset_index(drop=True)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Tokenize and pad the text sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

max_sequence_length = max(len(seq) for seq in X_train_sequences) + 1
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

models = {}
for col in y.columns:
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(label_encoders[col].classes_.shape[0], activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train_padded, y_train[col], validation_data=(X_test_padded, y_test[col]), epochs=3, batch_size=32)
    models[col] = model

medicine_name = text_after_word
medicine_name_sequence = tokenizer.texts_to_sequences([medicine_name])
medicine_name_padded = pad_sequences(medicine_name_sequence, maxlen=max_sequence_length)

predicted_purposes_decoded = {}
for col, model in models.items():
    predicted_purposes = model.predict(medicine_name_padded)
    predicted_purposes_decoded[col] = label_encoders[col].inverse_transform(predicted_purposes.argmax(axis=1))
    
engine = pyttsx3.init()
engine.setProperty('rate', 150)

for col, predicted_values in predicted_purposes_decoded.items():
    text =f'the {col} is, {predicted_values[0]}'; 
    # Use the engine to speak the text
    engine.say(text)
    engine.runAndWait()
    print(col + ": " + predicted_values[0])


Epoch 1/3
337/337 [==============================] - 12s 29ms/step - loss: 0.0995 - accuracy: 0.9825 - val_loss: 0.0937 - val_accuracy: 0.9800
Epoch 2/3
337/337 [==============================] - 8s 24ms/step - loss: 0.0271 - accuracy: 0.9916 - val_loss: 0.7239 - val_accuracy: 0.6606
Epoch 3/3
337/337 [==============================] - 8s 23ms/step - loss: 0.0037 - accuracy: 0.9992 - val_loss: 0.9192 - val_accuracy: 0.6610
Epoch 1/3
337/337 [==============================] - 28s 77ms/step - loss: 9.5539 - accuracy: 0.0000e+00 - val_loss: 9.6448 - val_accuracy: 0.0000e+00
Epoch 2/3
337/337 [==============================] - 26s 76ms/step - loss: 9.4552 - accuracy: 9.2739e-05 - val_loss: 9.9005 - val_accuracy: 0.0000e+00
Epoch 3/3
337/337 [==============================] - 26s 76ms/step - loss: 9.1346 - accuracy: 2.7822e-04 - val_loss: 10.3832 - val_accuracy: 0.0000e+00
Epoch 1/3
337/337 [==============================] - 11s 27ms/step - loss: 1.5936 - accuracy: 0.6547 - val_loss: 1.2572